In [1]:
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import mysql.connector
from unidecode import unidecode

In [2]:
df_matricial_tecnologias_tecnoempleo = pd.read_csv("../tecnologias_matricial_tecnoempleo.csv")
df_matricial_habilidades_tecnoempleo = pd.read_csv("../habilidades_matricial_tecnoempleo.csv")
df_matricial_habilidades_manfred = pd.read_csv("../habilidades_matricial_manfred.csv")
df_matricial_tecnologias_manfred = pd.read_csv("../programas_matricial_manfred.csv")
df_general = pd.read_csv("../ofertas_final.csv")

In [3]:
df_tecnologias_combinado = pd.concat([df_matricial_tecnologias_tecnoempleo, df_matricial_tecnologias_manfred], axis=0, join='outer', ignore_index=True)
df_habilidades_combinado = pd.concat([df_matricial_habilidades_tecnoempleo, df_matricial_habilidades_manfred], axis=0, join='outer', ignore_index=True)
df_matricial_hab_tec = pd.merge(df_tecnologias_combinado, df_habilidades_combinado, on='id',how='outer')
df_matricial_hab_tec = df_matricial_hab_tec.fillna(0)

In [4]:
df_general_codificado = pd.DataFrame()
df_general_codificado["id"] = df_general["id"]
df_general_codificado["salario_media"] = (df_general["salario_desde"] + df_general["salario_hasta"]) / 2
df_general_codificado["variable"] = df_general["variable"].fillna(0)
df_general_codificado["variable_booleano"] = df_general_codificado["variable"].apply(lambda x: 1 if x != 0 else 0)
moda_vacaciones = df_general["vacaciones"].mode()[0]
df_general_codificado["vacaciones"] = df_general["vacaciones"].fillna(moda_vacaciones) 
df_general_codificado["teletrabajo"] = df_general["teletrabajo"].map({"Presencial": 0, "Hibrido": 1, "Remoto": 2})
df_general_codificado["jornada_tipo"] = df_general["jornada_tipo"].map({"Intensiva Mañana": 0, "Intensiva Tarde": 1, "Jornada Completa": 2, 
                                                                "Jornada Completa Flexible": 3, "Media Jornada": 4, "Por Horas": 5, "Turno Rotatorio" : 6})
categorias_funciones = {cat: i for i, cat in enumerate(df_general["funciones"].dropna().unique())}
df_general_codificado["funciones"] = df_general["funciones"].map(categorias_funciones)
df_general_codificado["contrato_tipo"] = df_general["contrato_tipo"].map({"Indefinido": 0, "Temporal": 1, "Obra o servicio": 2,
                                                                  "Freelance/Autónomo": 3, "Prácticas": 4, "A determinar": 5})
df_general_codificado["nivel_profesional"] = df_general["nivel_profesional"].map({"Administrador / Consejero": 0, "Director / Gerente": 1, 
                                                                          "Mando Intermedio": 2, "Especialista": 3, "Empleado": 4, "Prácticas / Beca": 5})
df_general_codificado["formacion_minima"] = df_general["formacion_minima"].map({"Doctorado": 0, "Ingeniero Superior": 1, "Ingeniero Técnico": 2, "Licenciado": 3, 
                                                                        "Postgrado EEES (Máster)": 4, "Diplomado": 5, "FP2/Grado Superior": 6, 
                                                                        "Grado EEES (Bolonia)": 7, "Grado Medio": 8, "Bachillerato/COU": 9, 
                                                                        "E.S.O. (Educación Secundaria Obligatoria)": 10, "FP1": 11, "Sin estudios": 12, 
                                                                        "Otros títulos, certificaciones y carnets": 13, "Certificado de Profesionalidad": 14, 
                                                                        "Otra Formación Tecnológica": 15})

df_general_codificado["personas_a_cargo"] = df_general["personas_a_cargo"].fillna(0)
df_general_codificado["experiencia"] = df_general["experiencia"].apply(lambda x: 0 if x == "Sin Experiencia" 
                                                               else 1 if x in ["Menos de un año", "1 año"]
                                                               else 2 if x in ["2 años", "3 años", "3-5 años"]
                                                               else 3)

In [5]:
df_codificado = pd.merge(df_general_codificado, df_matricial_hab_tec, on='id',how='outer')

In [6]:
imputer = KNNImputer(n_neighbors=5)
columnas = df_codificado.drop(columns=["id"])
df_columnas_imputadas = imputer.fit_transform(columnas)
df_columnas_imputadas = pd.DataFrame(df_columnas_imputadas, columns=columnas.columns)
df_imputado = pd.concat([df_codificado["id"], df_columnas_imputadas], axis=1)
df_imputado.to_csv("/home/bosser/Escritorio/proyectofinal/df_imputado_plus.csv", index = False)